This notebook demonstrates an AI-powered X (Twitter) post generator using LangGraph and LangChain to show iterative workflow in langgraph. It employs multiple LLMs for generating, evaluating, and optimizing tweets through an iterative workflow, ensuring high-quality and engaging social media content.

In [1]:
from langgraph.graph import StateGraph,START, END
from typing import TypedDict, Literal, Annotated
from langchain_openai import ChatOpenAI
from langchain_core.messages import SystemMessage, HumanMessage
import operator

In [2]:
# based on capabilities, we can use different LLMs for different tasks

generator_llm = ChatOpenAI(model='gpt-4o-mini')
evaluator_llm = ChatOpenAI(model='gpt-4o')
optimizer_llm = ChatOpenAI(model='gpt-4o-mini')

In [3]:
# Create schema for structured output

from pydantic import BaseModel, Field

class TweetEvaluation(BaseModel):
    evaluation: Literal["approved", "needs_improvement"] = Field(..., description="Final evaluation result.")
    feedback: str = Field(..., description="feedback for the tweet.")

In [4]:
structured_evaluator_llm = evaluator_llm.with_structured_output(TweetEvaluation)

In [5]:
# state
class TweetState(TypedDict):

    topic: str
    tweet: str
    evaluation: Literal["approved", "needs_improvement"]
    feedback: str
    iteration: int
    max_iteration: int

    tweet_history: Annotated[list[str], operator.add]
    feedback_history: Annotated[list[str], operator.add]

In [6]:
def generate_tweet(state: TweetState):
    messages = [
        SystemMessage(content="You are a funny and clever Twitter/X influencer."),
        HumanMessage(content=f"""
                    Write a short, original, and hilarious tweet on the topic: "{state['topic']}".

                    Rules:
                    - Do NOT use question-answer format.
                    - Max 280 characters.
                    - Use observational humor, irony, sarcasm, or cultural references.
                    - Think in meme logic, punchlines, or relatable takes.
                    - Use simple, day to day english
                    """)
            ]
    response = generator_llm.invoke(messages).content
    return {'tweet': response, 'tweet_history': [response]}

In [7]:
def evaluate_tweet(state: TweetState):
    messages = [
    SystemMessage(content="You are a ruthless, no-laugh-given Twitter critic. You evaluate tweets based on humor, originality, virality, and tweet format."),
    HumanMessage(content=f"""
Evaluate the following tweet:

Tweet: "{state['tweet']}"

Use the criteria below to evaluate the tweet:

1. Originality – Is this fresh, or have you seen it a hundred times before?  
2. Humor – Did it genuinely make you smile, laugh, or chuckle?  
3. Punchiness – Is it short, sharp, and scroll-stopping?  
4. Virality Potential – Would people retweet or share it?  
5. Format – Is it a well-formed tweet (not a setup-punchline joke, not a Q&A joke, and under 280 characters)?

Auto-reject if:
- It's written in question-answer format (e.g., "Why did..." or "What happens when...")
- It exceeds 280 characters
- It reads like a traditional setup-punchline joke
- Dont end with generic, throwaway, or deflating lines that weaken the humor (e.g., “Masterpieces of the auntie-uncle universe” or vague summaries)

### Respond ONLY in structured format:
- evaluation: "approved" or "needs_improvement"  
- feedback: One paragraph explaining the strengths and weaknesses 
""")
]

    response = structured_evaluator_llm.invoke(messages)

    return {'evaluation':response.evaluation, 'feedback': response.feedback, 'feedback_history': [response.feedback]}

In [8]:
def optimize_tweet(state: TweetState):

    messages = [
        SystemMessage(content="You punch up tweets for virality and humor based on given feedback."),
        HumanMessage(content=f"""
Improve the tweet based on this feedback:
"{state['feedback']}"

Topic: "{state['topic']}"
Original Tweet:
{state['tweet']}

Re-write it as a short, viral-worthy tweet. Avoid Q&A style and stay under 280 characters.
""")
    ]

    response = optimizer_llm.invoke(messages).content
    iteration = state['iteration'] + 1

    return {'tweet': response, 'iteration': iteration, 'tweet_history': [response]}

In [9]:
def route_evaluation(state: TweetState):

    if state['evaluation'] == 'approved' or state['iteration'] >= state['max_iteration']:
        return 'approved'
    else:
        return 'needs_improvement'

In [10]:
graph = StateGraph(TweetState)

graph.add_node('generate', generate_tweet)
graph.add_node('evaluate', evaluate_tweet)
graph.add_node('optimize', optimize_tweet)

graph.add_edge(START, 'generate')
graph.add_edge('generate', 'evaluate')

# if approved send to END else send to optimize node
graph.add_conditional_edges('evaluate', route_evaluation, {'approved': END, 'needs_improvement': 'optimize'})
graph.add_edge('optimize', 'evaluate')

workflow = graph.compile()

In [11]:
# visualize
try:
    from IPython.display import Image
    Image(workflow.get_graph().draw_mermaid_png())
except:
    pass

In [12]:
# we can visualize the graph using mermaid syntax
# https://mermaid.live/
# sometime due to network issues, image may not render, in that case use mermaid_code below to visualize

mermaid_code = workflow.get_graph().draw_mermaid()
print(mermaid_code)

---
config:
  flowchart:
    curve: linear
---
graph TD;
	__start__([<p>__start__</p>]):::first
	generate(generate)
	evaluate(evaluate)
	optimize(optimize)
	__end__([<p>__end__</p>]):::last
	__start__ --> generate;
	evaluate -. &nbsp;approved&nbsp; .-> __end__;
	evaluate -. &nbsp;needs_improvement&nbsp; .-> optimize;
	generate --> evaluate;
	optimize --> evaluate;
	classDef default fill:#f2f0ff,line-height:1.2
	classDef first fill-opacity:0
	classDef last fill:#bfb6fc



In [13]:
initial_state = {
    "topic": "Egdbeq",
    "iteration": 1,
    "max_iteration": 5
}
final_state = workflow.invoke(initial_state)

In [14]:
final_state

{'topic': 'Egdbeq',
 'tweet': 'My keyboard this Monday be like: "I can’t type without coffee!" ☕️ If it keeps this up, I might have to send it to therapy or just get it a double shot! #MondayMood #CaffeineConfessions',
 'evaluation': 'needs_improvement',
 'feedback': "The tweet attempts to play off the relatable theme of needing coffee on a Monday morning, a concept that is quite overused, diminishing its originality. While it aims for humor, the joke about sending the keyboard to therapy or giving it a double shot falls flat and doesn't quite land a punchy impact or evoke a strong reaction. The format is acceptable as it doesn't follow a setup-punchline structure and fits within the character limit, but the ending hashtags feel generic and don't add significant value to increase its virality potential. Overall, although the structure is sound, the content requires more creativity to stand out.",
 'iteration': 5,
 'max_iteration': 5,
 'tweet_history': ['Just found out "Egdbeq" is not a